In [2]:
from sklearn.model_selection import KFold
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, pipeline
import pandas as pd

In [29]:
dataset = load_dataset(
    'csv',
    data_files={'train': './datas/training_text_personal_data.csv'},
    delimiter=';',
    )


Generating train split: 52 examples [00:00, 642.26 examples/s]


In [30]:
MODEL_NAME = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
def preprocess(data):
    return tokenizer(data['text'], padding='max_length', truncation=True, max_length=512)

In [32]:
tokenized_dataset = dataset['train'].map(preprocess, batched=True)

Map: 100%|██████████| 52/52 [00:00<00:00, 771.43 examples/s]


In [33]:
print(type(tokenized_dataset))

<class 'datasets.arrow_dataset.Dataset'>


In [34]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [35]:
results = []

In [36]:
for fold, (train_index, test_index) in enumerate(kf.split(tokenized_dataset)):
    print(f"Treinando o Fold {fold + 1}")
    
    train_data = tokenized_dataset.select(train_index)
    test_data = tokenized_dataset.select(test_index)
    
    training_args = TrainingArguments(
        output_dir=f"./datas/results_fold_{fold + 1}",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        num_train_epochs=3,
        save_total_limit=1,
        logging_dir=f"./logs_fold_{fold + 1}",
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data,
        eval_dataset=test_data,
        tokenizer=tokenizer,
    )
    
    trainer.train()
    
    eval_result = trainer.evaluate()
    results.append(eval_result)
    print(f"Resultados para o Fold {fold + 1}: {eval_result}")

Treinando o Fold 1


c:\Users\iqued\PUC_rio\mvp_machine_learning_and_analytics\venv\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\iqued\AppData\Local\Temp\ipykernel_7900\2385051996.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
 33%|███▎      | 6/18 [01:49<02:50, 14.22s/it]



                                       
                                              
 33%|███▎      | 6/18 [01:56<02:50, 14.22s/it]


{'eval_loss': 1.0028258562088013, 'eval_runtime': 6.4798, 'eval_samples_per_second': 1.698, 'eval_steps_per_second': 0.309, 'epoch': 1.0}


 67%|██████▋   | 12/18 [03:33<01:22, 13.72s/it]



                                       
                                               
 67%|██████▋   | 12/18 [03:39<01:22, 13.72s/it]


{'eval_loss': 0.9608098864555359, 'eval_runtime': 6.4826, 'eval_samples_per_second': 1.697, 'eval_steps_per_second': 0.309, 'epoch': 2.0}


...

{'eval_loss': 0.22464656829833984, 'eval_runtime': 5.2799, 'eval_samples_per_second': 1.894, 'eval_steps_per_second': 0.379, 'epoch': 2.0}


100%|██████████| 18/18 [04:51<00:00, 14.00s/it]



                                       
                                               
100%|██████████| 18/18 [05:00<00:00, 14.00s/it]

100%|██████████| 18/18 [05:00<00:00, 16.70s/it]


{'eval_loss': 0.20889540016651154, 'eval_runtime': 4.8407, 'eval_samples_per_second': 2.066, 'eval_steps_per_second': 0.413, 'epoch': 3.0}
{'train_runtime': 300.6342, 'train_samples_per_second': 0.419, 'train_steps_per_second': 0.06, 'train_loss': 0.2668522728814019, 'epoch': 3.0}


100%|██████████| 2/2 [00:01<00:00,  1.99it/s]

Resultados para o Fold 5: {'eval_loss': 0.20889540016651154, 'eval_runtime': 5.0383, 'eval_samples_per_second': 1.985, 'eval_steps_per_second': 0.397, 'epoch': 3.0}


In [37]:
print("Resultados de Cross-Validation:", results)

Resultados de Cross-Validation: [{'eval_loss': 0.9470769762992859, 'eval_runtime': 6.2071, 'eval_samples_per_second': 1.772, 'eval_steps_per_second': 0.322, 'epoch': 3.0}, {'eval_loss': 0.8099384307861328, 'eval_runtime': 6.947, 'eval_samples_per_second': 1.583, 'eval_steps_per_second': 0.288, 'epoch': 3.0}, {'eval_loss': 0.5600932240486145, 'eval_runtime': 5.6719, 'eval_samples_per_second': 1.763, 'eval_steps_per_second': 0.353, 'epoch': 3.0}, {'eval_loss': 0.47197189927101135, 'eval_runtime': 5.45, 'eval_samples_per_second': 1.835, 'eval_steps_per_second': 0.367, 'epoch': 3.0}, {'eval_loss': 0.20889540016651154, 'eval_runtime': 5.0383, 'eval_samples_per_second': 1.985, 'eval_steps_per_second': 0.397, 'epoch': 3.0}]


In [38]:
eval_losses = [0.9471, 0.8099, 0.5601, 0.4720, 0.2089]
mean_loss = sum(eval_losses) / len(eval_losses)
print(f"Média do eval_loss: {mean_loss}")

Média do eval_loss: 0.5996


In [44]:
final_training_args = TrainingArguments(
    output_dir="./final_model",
    eval_strategy="no",  # Desativar avaliação, pois a validação cruzada já foi feita
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    save_total_limit=1,
    logging_dir="./final_logs",
)

In [46]:
final_trainer = Trainer(
    model=model,
    args=final_training_args,
    train_dataset=tokenized_dataset, 
    processing_class=tokenizer,
)


In [47]:
final_trainer.train()

100%|██████████| 21/21 [06:28<00:00, 18.51s/it]

{'train_runtime': 388.7701, 'train_samples_per_second': 0.401, 'train_steps_per_second': 0.054, 'train_loss': 0.12189194134303502, 'epoch': 3.0}


TrainOutput(global_step=21, training_loss=0.12189194134303502, metrics={'train_runtime': 388.7701, 'train_samples_per_second': 0.401, 'train_steps_per_second': 0.054, 'total_flos': 41045693165568.0, 'train_loss': 0.12189194134303502, 'epoch': 3.0})

In [48]:
model.save_pretrained("./final_model")
tokenizer.save_pretrained("./final_model")

print("Modelo final treinado e salvo em './final_model'")

Modelo final treinado e salvo em './final_model'
